In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import scipy
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from tqdm import tqdm
from scipy.linalg.lapack import zggev
from scipy.linalg import block_diag
import pandas as pd
from sklearn.cross_decomposition import CCA, PLSCanonical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from direct_effect_analysis import * 
from utils import *
from mvlearn.embed import GCCA
from pgmpy.estimators import PC
from plotnine import ggplot, aes, geom_line, geom_ribbon, scale_x_log10, scale_x_continuous, labs, theme, facet_wrap, ggsave
from sklearn.ensemble import RandomForestRegressor

In [2]:
algorithms = ['PCA', 'genCCA', 'OptiDet', 'Roy', 'RoyS']
B_conds = ['d', '1', '1/d', '1/d^2']
Sigma_conds = ['d', '1', '1/d', '1/d^2']
results = {B_cond:{Sigma_cond:{algo: {} for algo in algorithms } for Sigma_cond in Sigma_conds} for B_cond in B_conds} 

In [3]:
p, r, d, q, rk = 10, 10, 100, 1, 10
N = 1000
dimensions = [2, 5, 10, 20, 50, 100, 200]

In [4]:
a, b, c = 0.1, 0.1, 0.8
B = 1
for B_cond in tqdm(B_conds):
    for Sigma_cond in Sigma_conds:
        for algo in algorithms:
            # print(noise, algo)
            CORRS = []
            for d in dimensions:
                d = int(d)
                CORR = []
                for j in range(B):

                    # Generate parameters
                    _, A_z, beta, gamma = parameters(r, p, q, d, False)

                    if B_cond == 'd':
                        A_x = np.array([i for i in range(d)])[:, None].T
                    elif B_cond == '1':
                        A_x = np.array([1 for i in range(d)])[:, None].T
                    elif B_cond == '1/d':
                        A_x = np.array([1/(i+1) for i in range(d)])[:, None].T
                    elif B_cond == '1/d^2':
                        A_x = np.array([1/((i+1)**2) for i in range(d)])[:, None].T

                    if Sigma_cond == 'd':
                        Sigma = np.diag(np.array([i for i in range(d)]))
                    elif Sigma_cond == '1':
                        Sigma = np.identity(d)
                    elif Sigma_cond == '1/d':
                        Sigma = np.diag(np.array([1/(i+1) for i in range(d)]))
                    elif Sigma_cond == '1/d^2':
                        Sigma = np.diag(np.array([1/((i+1)**2) for i in range(d)]))


                    # Generate training and test data
                    X_train, Y_train, Z_train, Y_x_train = generate_data_Sigma_nonlinear(N, p, r, d, beta, gamma, A_x, A_z, Sigma, a=a, b=b, c=c, nl_param=2)
                    # Generate training and test data
                    dea = DirectEffectAnalysis(type=algo, alpha=1e-5, regressor_0=RandomForestRegressor(n_estimators=100), regressor_1=RandomForestRegressor(n_estimators=100))
                    dea.fit(X_train, Y_train, Z_train)
                    Y_hat = dea.transform(X_train, Y_train, Z_train)
                    corr = np.abs(np.corrcoef(Y_hat, Y_x_train[:,0])[0, 1])
                    
                    # Compute MSE for each iteration
                    CORR.append(corr)
                CORRS.append(CORR)
            # Append MSE for current sample size
            results[B_cond][Sigma_cond][algo] = CORRS

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Assuming you have the following data structures
algo_names = {'PCA':'PCA', 'genCCA':'T_C', 'OptiDet':'T_D', 'Roy':'T_F', 'RoyS':'T_S'}
# results, B_conds, Sigma_conds, algorithms, dimensions are defined elsewhere

# Prepare data for ggplot
data = []
for B_cond in B_conds:
    for Sigma_cond in Sigma_conds:
        for algo in algorithms:
            algo_data = results[B_cond][Sigma_cond][algo]
            values = np.array(algo_data)
            median = np.percentile(values, 50, axis=1)
            lower = np.percentile(values, 5, axis=1)
            upper = np.percentile(values, 95, axis=1)
            
            for i, dim in enumerate(dimensions):
                data.append({
                    'Dimension': dim,
                    'Median': median[i],
                    'Lower': lower[i],
                    'Upper': upper[i],
                    'Algorithm': algo_names[algo],
                    'Sigma': Sigma_cond,
                    'B': B_cond  # Add weight name to the data
                })

# Create a DataFrame
df = pd.DataFrame(data)
df2 = df[df['Algorithm']!='PCA']

# Create a new column for the facet label
df2['Facet_Label'] = 'Sigma: Diag(...,' + df2['Sigma'].astype(str) + '), B: [...,' + df2['B'].astype(str) + ']'

# Plot using ggplot with logarithmic scale on the x-axis
p = (ggplot(df2, aes(x='Dimension', y='Median', color='Algorithm', fill='Algorithm'))
     + geom_line(size=1.5)
     + geom_ribbon(aes(ymin='Lower', ymax='Upper'), alpha=0.2)
     + scale_x_log10(breaks=dimensions)  # Set specific x-axis ticks to log scale
     + labs(x='Dimension (d)', y='Correlation')
     + facet_wrap('~Facet_Label', ncol=4)  # Use the new Facet_Label for facets
     + theme(legend_position='right', figure_size=(12, 8))  # Set figure size
)

# Display the plot
print(p)

/tmp/ipykernel_889/2443074691.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_889/2443074691.py:45: FutureWarning: Using print(plot) to draw and show the plot figure is deprecated and will be removed in a future version. Use plot.show().
INFO:matplotlib.font_manager:Fontsize 0.00 < 1.0 pt not allowed by FreeType. Setting fontsize = 1 pt
